In [1]:
from question_generation import data_utils
from onmt.bin.translate import translate
from onmt.bin.translate import _get_parser
from transformers import GPT2Tokenizer


/local/miniconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/local/miniconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/local/miniconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/local/miniconda3/envs/python37/lib/python3.7/site-pack

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('/home/mroemmele/question_generation/gpt2_tokenizer_vocab/')

In [3]:
'''Load texts as input to QG'''

input_texts = [text.strip() for text in open("/home/mroemmele/question_generation/qg_inputs_outputs/inputs/nmt_blog_concat.segmented.txt")]
input_texts[:5]

['\ufeffSDL (LSE: SDL), a leader in global content management, translation and digital experience, today announced that its next-generation SDL Neural Machine Translation (NMT) 2.0 has mastered Russian to English translation, one of the toughest linguistic Artificial Intelligence (AI) problems to date.',
 'SDL NMT 2.0 outperformed all industry standards, setting a benchmark for Russian to English machine translation, with over 90% of the system’s output labelled as perfect by professional Russian-English translators.',
 'The new SDL NMT 2.0 Russian engine is being made available to enterprise customers via SDL Enterprise Translation Server (ETS), a secure NMT product, enabling organizations to translate large volumes of information into multiple languages.',
 '“One of the toughest linguistic challenges facing the machine translation community has been overcome by our team,” said Adolfo Hernandez, CEO, SDL.',
 '“It was the Russian language that first inspired the science and research be

In [4]:
'''Tokenize input texts'''

tokenized_texts = [" ".join(data_utils.tokenize_fn(tokenizer, text)) for text in input_texts]
tokenized_texts[:10]

['ï » ¿ SD L Ġ( L SE : ĠSDL ), Ġa Ġleader Ġin Ġglobal Ġcontent Ġmanagement , Ġtranslation Ġand Ġdigital Ġexperience , Ġtoday Ġannounced Ġthat Ġits Ġnext - generation ĠSDL ĠNeural ĠMachine ĠTranslation Ġ( N MT ) Ġ2 . 0 Ġhas Ġmastered ĠRussian Ġto ĠEnglish Ġtranslation , Ġone Ġof Ġthe Ġtoughest Ġlinguistic ĠArtificial ĠIntelligence Ġ( AI ) Ġproblems Ġto Ġdate .',
 'SD L ĠN MT Ġ2 . 0 Ġoutper formed Ġall Ġindustry Ġstandards , Ġsetting Ġa Ġbenchmark Ġfor ĠRussian Ġto ĠEnglish Ġmachine Ġtranslation , Ġwith Ġover Ġ90 % Ġof Ġthe Ġsystem âĢ Ļ s Ġoutput Ġlabelled Ġas Ġperfect Ġby Ġprofessional ĠRussian - English Ġtransl ators .',
 'The Ġnew ĠSDL ĠN MT Ġ2 . 0 ĠRussian Ġengine Ġis Ġbeing Ġmade Ġavailable Ġto Ġenterprise Ġcustomers Ġvia ĠSDL ĠEnterprise ĠTranslation ĠServer Ġ( ETS ), Ġa Ġsecure ĠN MT Ġproduct , Ġenabling Ġorganizations Ġto Ġtranslate Ġlarge Ġvolumes Ġof Ġinformation Ġinto Ġmultiple Ġlanguages .',
 'âĢ ľ One Ġof Ġthe Ġtoughest Ġlinguistic Ġchallenges Ġfacing Ġthe Ġmachine Ġtranslat

In [32]:
len(tokenized_texts)

51

In [5]:
'''Save tokenized input texts'''

with open("/home/mroemmele/question_generation/qg_inputs_outputs/inputs/nmt_blog_concat.segmented.txt.gpt2_tok", "w") as f:
    f.write("\n".join(tokenized_texts))

In [41]:
'''Run QG'''

parser = _get_parser()
opt = parser.parse_args(args=['-model', '/home/mroemmele/question_generation/squad_newsqa_gpt2_tok_no_masking_models/finetuned_rule_mimic/trained_models/model_step_5600.pt',
                              #                               '-src', "/home/mroemmele/question_generation/qg_inputs_outputs/inputs/nmt_blog_concat.segmented.txt.gpt2_tok",
                              '-src', "",
                              #                               '-gpu', '0',
                              '-batch_size', '65536',#'1024', #,
                              '-batch_type', 'tokens',
                              #                               '-random_sampling_topk', '500',
                              #                               '-coverage_penalty', 'wu',
                              #                               '-beta', '0.8',
                              # #                               '-random_sampling_topk', '1',
                              '-beam_size', '5',
                              '-block_ngram_repeat', '1',
                              #                               '-shard_size', '100000',
                              #                               '-random_sampling_temp', '0.2',
                              # '-output', "/home/mroemmele/question_generation/qg_inputs_outputs/outputs/nmt_blog_concat.segmented.txt.gpt2_tok"]
                              ])

In [42]:
from onmt.translate.translator import build_translator
translator = build_translator(opt, report_score=True)

In [43]:
inputs = [text.encode() for text in tokenized_texts[:opt.shard_size]]

In [44]:

    
scores, gen_qs = translator.translate(
        src=inputs,
        batch_size=opt.batch_size,
        batch_type=opt.batch_type,
#         attn_debug=opt.attn_debug,
#         align_debug=opt.align_debug
        )

PRED AVG SCORE: -0.8206, PRED PPL: 2.2718


In [45]:
detok_gen_qs = [data_utils.detokenize_fn(tokenizer, gen_q[0].strip().split(" ")) for gen_q in gen_qs]
detok_gen_qs

['What does the toughest linguistic Artificial Intelligence mean?',
 'What is the focus of NMT 2.0?',
 'What is the new SDL NMT 2.0?',
 'What is the toughest linguistic challenges?',
 'What was the Russian language?',
 'What is the global expansion of the enterprise customers?',
 'What are the names of the Russian language?',
 'What is Russian a highly inflected language with?',
 'What is the focus of the translation quality?',
 'What did Samad Echihabi say?',
 'What is the reason for phenomenal technology?',
 'What is SDL NMT 2.0?',
 'What is the data trained for?',
 'What is NMT 2.0 developed with?',
 'What is the level of productivity to Language?',
 'What is NMT 2.0?',
 'Why has ETS been used?',
 'What does SDLL mean?',
 'What does Edge-Cloud capabilities use?',
 'What does NMT 2.0 solution to?',
 'Why do leveraging on-cloud instant translation?',
 'What are some language pairs of SDL?',
 'Why can brands develop their business?',
 'What is instant access from anywhere?',
 'Why are 

In [ ]:
'''Load QG output'''
gen_qs = [question.strip().split(" ") for question in open("/home/ec2-user/qg_inputs_outputs/outputs/nmt_blog_concat.segmented.txt.gpt2_tok.topk")]
detok_gen_qs = [data_utils.detokenize_fn(tokenizer, gen_q) for gen_q in gen_qs]
detok_gen_qs